## 修改数据集标注

In [32]:
import os
from xml.etree import ElementTree as ET
import shutil
from tqdm import tqdm 
import os
import random

def makedirifnoExist(path):
    if(not os.path.exists(path)):
        os.makedirs(path)

In [10]:
print(os.listdir("/root/dataset/xuelang7"))

['norm', '.DS_Store', 'Annatations', 'defect_3', 'defect_7', 'defect_6', 'defect_5', 'JPEGImages', 'defect_4', 'defect_2', 'ImageSets', 'defect_1']


In [35]:
path = "/root/dataset/xuelangplus1"

In [37]:

JPEGImages = os.path.join(path, "JPEGImages")
Annatations = os.path.join(path, "Annatations")
ImageSets = os.path.join(path, "ImageSets")
makedirifnoExist(JPEGImages)
makedirifnoExist(Annatations)
makedirifnoExist(ImageSets)

for folder in tqdm(os.listdir(path)):
    if folder[0] == ".":
        continue
    folder_path = os.path.join(path, folder)
    for file in os.listdir(folder_path):
        if(file[0]=="."):
            continue
        file_path = os.path.join(folder_path, file)
        if(file[-3:]=="xml" and not os.path.isfile(os.path.join(Annatations,file))): 
            xml_path = os.path.join(folder_path, file)
            jpg_path = os.path.join(folder_path, file[:-3]+"jpg")
            
            updateTree = ET.parse(xml_path)
            annotation = updateTree.getroot()
            newEle = ET.Element("folder")
            newEle.text = "xuelang"
            annotation.append(newEle)
            #<folder>xuelang</folder>
            obj = annotation.find("object")
            obj_name = obj.find("name")
            obj_name.text = folder
            updateTree = updateTree.write(xml_path)
            shutil.copy(xml_path, Annatations)
            shutil.copy(jpg_path, JPEGImages)

        

100%|██████████| 4/4 [00:00<00:00, 372.91it/s]


In [36]:
trainval_percent = 0.9
train_percent = 0.7

xmlfilepath = os.path.join(path,"Annotations")
txtsavepath = os.path.join(path, "ImageSets/Main")

trainval = []
train = []
val = []
test = []

for folder in os.listdir(path):
    if(folder == "norm" or folder[0] == "."):
        continue
    folder_path = os.path.join(path, folder)
    xml_list = []
    for file in os.listdir(folder_path):
        if(file[-3:]=="xml"):
            xml_list.append(file[:-4]+"\n")
    random.shuffle(xml_list)
    l = len(xml_list)
    trainval += xml_list[:int(l*trainval_percent)]
    train += xml_list[:int(l*train_percent)]
    val += xml_list[int(l*train_percent):int(l*trainval_percent)]
    test += xml_list[int(l*trainval_percent):]

makedirifnoExist(os.path.join(path,"ImageSets/Main"))

ftrainval = open(os.path.join(path,"ImageSets/Main/trainval.txt"), 'w')
ftest = open(os.path.join(path,"ImageSets/Main/test.txt"), 'w')
ftrain = open(os.path.join(path,"ImageSets/Main/train.txt"), 'w')
fval = open(os.path.join(path,"ImageSets/Main/val.txt"), 'w')


list_txt = [[trainval,ftrainval],[train,ftrain],[val,fval],[test,ftest]]


for tlist, txt  in list_txt:
    
    for filename in tlist:
        txt.write(filename)
    txt.close()
    
ftrainval.close()
ftrain.close()
fval.close()
ftest .close()

In [47]:
def Splitdataset(traintxt, savedir, annodir=None, imgdir=None):
    annolist = []
    for i in os.listdir(annodir):
        if i[0]==".":
            continue
        annolist.append(i[:-4])
    #打开train.txt文件，读取训练集文件名(不含后缀)
    f = open(traintxt, 'r')
    trainlist =[file.replace('\n','') for file in f.readlines()]
    f.close()
    #分别建立训练和测试集图片和标注文件的文件夹，一共4个
    train_xml_path = os.path.join(savedir, 'trainAnnotations')
    train_img_path = os.path.join(savedir, 'trainJPEGImages')
    test_xml_path = os.path.join(savedir, 'testAnnotations')
    test_img_path = os.path.join(savedir, 'testJPEGImages')
    makedirifnoExist(train_xml_path)
    makedirifnoExist(train_img_path)
    makedirifnoExist(test_xml_path)
    makedirifnoExist(test_img_path)
    #遍历训练集文件名
    for i, xml in enumerate(trainlist):
        #将训练集中的图片和标注文件复制出来
        #xml = xml.replace('\n', '')
        shutil.copyfile(os.path.join(annodir, xml + '.xml' ), 
                    os.path.join(train_xml_path, xml+'.xml'))
        shutil.copyfile(os.path.join(imgdir, xml)+ '.jpg', 
                    os.path.join(train_img_path, xml) + '.jpg')
        #根据全部数据集文件名和训练集文件名，得出测试集文件名
        annolist.remove(xml)
            
            #遍历测试集文件名
    for i, xml in enumerate(annolist):
        #将测试集中的文件复制出来
        shutil.copyfile(os.path.join(annodir, xml+'.xml'), 
                    os.path.join(test_xml_path, xml+'.xml'))
        shutil.copyfile(os.path.join(imgdir, xml)+ '.jpg', 
                    os.path.join(test_img_path, xml) + '.jpg')

In [48]:
savedir = "/root/workspace/Cascade-RCNN_Tensorflow/data/xuelangplus1"
imgdir = "/root/dataset/xuelangplus1/JPEGImages"
annodir = "/root/dataset/xuelangplus1/Annatations"
train = "/root/dataset/xuelangplus1/ImageSets/Main/trainval.txt"
Splitdataset(train, savedir, annodir, imgdir)

In [17]:
f = open(train, 'r')
trainlist = f.readlines()
f.close()
res = {}
for i in trainlist:
    if i not in res.keys():
        res[i] = 1
    else:
        res[i] += 1
        print(res[i])

print(len(trainlist))

71


In [19]:
folder_path = "/root/dataset/xuelang/Annatations"
for file in os.listdir(folder_path):
    if(file[0]=="."):
        continue
    xml_path = os.path.join(folder_path, file)

    updateTree = ET.parse(xml_path)
    annotation = updateTree.getroot()
    newEle = ET.Element("folder")
    newEle.text = "xuelang"
    annotation.append(newEle)
    #<folder>xuelang</folder>
    objs = annotation.findall("object")
    for obj in objs:
        obj_name = obj.find("name")
        obj_name.text = "defect_1"
    updateTree = updateTree.write(xml_path)
        

# new datas

In [2]:
import os
from xml.etree import ElementTree as ET
import shutil
from tqdm import tqdm 
import os
import random

def makedirifnoExist(path):
    if(not os.path.exists(path)):
        os.makedirs(path)
        

In [28]:
def changeLable(xml_file_path, save_path,file):
    updateTree = ET.parse(xml_file_path)
    annotation = updateTree.getroot()
    obj = annotation.find("filename")
    obj.text = file
    updateTree = updateTree.write(save_path)
path = "/root/workspace/output"
xmlpath = "/root/dataset/xuelang7/Annatations"
imgpath = "/root/dataset/xuelang7/JPEGImages"
save_img_path = "/root/dataset/xuelangplus7/JPEGImages"
save_xml_path = "/root/dataset/xuelangplus7/Annatations"
for folder in os.listdir(path):
    folderpath = os.path.join(path,folder)
    for file in os.listdir(folderpath):
        filepath = os.path.join(folderpath,file)
        shutil.copy(filepath,save_img_path)
        filename = ""
        for nn in file.split("_")[:-1]:
            filename += nn + "_"
        xml_file_path = os.path.join(xmlpath, filename[:-1]+'.xml' )
        save_xml_file_path = os.path.join(save_xml_path, file[:-4]+'.xml' )
        changeLable(xml_file_path,save_xml_file_path,file)


In [29]:
from tqdm import tqdm
path1="/root/dataset/xuelangplus7/JPEGImages"
path2="/root/dataset/xuelang7/JPEGImages"
for file in tqdm(os.listdir(path2)):
    filepath =os.path.join(path2,file)
    shutil.copy(filepath,path1)

100%|██████████| 706/706 [00:24<00:00, 29.22it/s]


In [30]:
from tqdm import tqdm
path1="/root/dataset/xuelangplus7/Annatations"
path2="/root/dataset/xuelang7/Annatations"
for file in tqdm(os.listdir(path2)):
    filepath =os.path.join(path2,file)
    shutil.copy(filepath,path1)

100%|██████████| 706/706 [00:00<00:00, 8119.91it/s]


In [31]:
print(len(os.listdir(path1)))

3530


In [ ]:
changeLable(xml_file_path,save_path,file):
    updateTree = ET.parse(xml_path)
    annotation = updateTree.getroot()
    obj = annotation.findall("filename")
    obj.text = file
    updateTree = updateTree.write(save_path)